<a href="https://colab.research.google.com/github/Namyalg/Projects_in_python/blob/master/PGNfile_to_final_positions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

THIS is the code for reading any given pgn or portable game notattion file and then displaying the final moves in the format given below, it can also be convereted to fen 


Get the files from drive


In [0]:
import re

with open("pgn3.txt", "r") as file: 
  file_contents = ''
  for line in file:
    file_contents += line + ' '
  info = re.compile('<[^>]*>')
  file_contents = (file_contents.replace('\n' ,'').replace('[' ,'<').replace(']','>').replace('{' , '').replace('}' ,''))
  file_contents = info.sub('' , file_contents).strip()
  game = [i.strip() for i in file_contents.split('.')][1:]
  last_move = game[-1]
  result = re.compile('( *1 *- *0 *| *0 *- *1 *| *1/2 *- *1/2 *)')
  last_move = result.sub('' , last_move)
  game = game[:-1]
  if len([last_move])!= 0:
    game = [i.split()[:-1] for i in game] + [[last_move ,'']]
  else:
    game = [i.split()[:-1] for i in game]



  
_

[['d4', 'd5'], ['c4', 'c6'], ['Nf3', 'Nf6'], ['Nc3', 'dxc4'], ['a4', 'Bf5'], ['e3', 'e6'], ['Bxc4', 'Bb4'], ['O-O', 'Nbd7'], ['Qe2', 'Bg4'], ['h3', 'Bxf3'], ['Qxf3', 'O-O'], ['Rd1', 'Qa5'], ['e4', 'e5'], ['d5', 'Nb6'], ['Bb3', 'Bxc3'], ['bxc3', 'cxd5'], ['exd5', 'Rac8'], ['c4', 'Rfe8'], ['Bd2', 'Qa6'], ['d6', 'Nbd7'], ['Be3', 'Rc6'], ['a5', 'Rxd6'], ['Ba4', 'Rxd1+'], ['Rxd1', 'Rd8'], ['c5', 'h6'], ['Qf5', 'g6'], ['Qc2', 'Qxa5'], ['Rd6', 'Kg7'], ['Qd1', 'Qc7'], ['h4', 'Kh8'], ['h5', 'g5'], ['Qf3', 'Kg7'], ['Qf5', 'b6'], ['Bxd7', 'Nxd7'], ['Bxg5', '']]


''

In [0]:
def rank_diff(from_ , to_):
  return(int(to_[1]) - int(from_[1]))

def file_diff(from_, to_):
  return(ord(to_[0]) - ord(from_[0]))

def fr(from_ , to_):
  return(rank_diff(from_ , to_) , file_diff(from_ , to_))

def rook(from_ , to_):
  st = fr(from_ , to_)
  return(st[0] ==0  or st[1] == 0)

def bishop(from_ , to_):
  st = fr(from_ , to_)
  return(abs(st[0]) == abs(st[1]))

def knight(from_ , to_):
  st = fr(from_ , to_)
  return((abs(st[0]) , abs(st[1])) in [(1,2) , (2,1)])

def king(from_ , to_):
  st = fr(from_ , to_)
  return(abs(st[0]) , abs(st[1]) in [(1,0) , (0,1) , (1,1)])

def queen(from_ , to_):
  return(bishop(from_ , to_) or rook(from_ , to_))

def wPawn(from_ ,to_):
  st = fr(from_ , to_)
  if int(from_[1]) == 2:
    return(st[0] in [1,2] and st[1] == 0)
  return(st[0] == 1 and st[1] == 0)


def bPawn(from_ , to_):
  st = fr(from_ , to_)
  if int(from_[1]) == 7:
    return(abs(st[0]) in [1,2] and st[1] == 0)
  return(abs(st[0]) == 1 and st[1] == 0)

def wPawncap(from_ , to_):
  return(rank_diff(from_ , to_) == 1 and file_diff(from_ , to_) in [-1,1])

def bPawncap(from_ , to_):
  return(rank_diff(from_ , to_) == -1 and file_diff(from_ , to_) in [-1,1])

PAWN AND PIECE MOVEMENT

In [0]:
def white_pawn_moves(white , white_pawns):
  for pawn , from_ in white_pawns.items():
        if wPawn(from_ , white):
          white_pawns[pawn] = white
          break
  return(white_pawns)

def black_pawn_moves(black , black_pawns):
  for pawn , from_ in black_pawns.items():
       if bPawn(from_ , black):
         black_pawns[pawn] = black
         break
  return(black_pawns)

def white_piece_moves(white , white_pieces):
    which_one = {'K':king , 'Q':queen , 'N':knight , 'B':bishop , 'R' : rook}
    pc = white[0]
    white = white[1:]
    for piece , from_ in white_pieces.items():
      if piece.startswith(pc):
        if which_one[pc](from_ , white):
          temp = piece
    white_pieces[temp] = white
    return(white_pieces)


def black_piece_moves(black , black_pieces):
    which_one = {'K':king , 'Q':queen , 'N':knight , 'B':bishop , 'R' : rook}
    pc = black[0]
    black = black[1:]
    for piece, from_ in black_pieces.items():
      if piece.startswith(pc):
        if which_one[pc](from_ , black):
          temp = piece
    black_pieces[temp] = black
    return(black_pieces)


KING-SIDE AND QUEEN-SIDE CASTLING

In [0]:
def white_kings_castle(white_pieces):
    white_pieces['K'] = 'g1'
    white_pieces['R2'] = 'f1'
    return(white_pieces)

def white_queens_castle(white_pieces):
    white_pieces['K'] = 'c1'
    white_pieces['R1'] = 'd1'
    return(white_pieces)


def black_kings_castle(black_pieces):
  black_pieces['K'] = 'g8'
  black_pieces['R2'] = 'f8'
  return(black_pieces)


def black_queens_castle(black_pieces):
    black_pieces['K'] = 'c8'
    black_pieces['R1'] = 'd8'
    return(black_pieces)
  
    


AMBIGUOUS MOVES


In [0]:
def black_ambiguous(black , black_pieces):
    pc = black[0]
    on_pos = black[1]
    to_pos = black[2:]
    which_one = {'R' : rook , 'K':king , 'Q':queen , 'N':knight , 'B':bishop}
    for piece, from_ in black_pieces.items():
      if piece.startswith(pc) and from_.startswith(on_pos):
        if which_one[pc](from_ , to_pos):
          black_pieces[piece] = to_pos
          break
    return(black_pieces)

def white_ambiguous(white , white_pieces):
    pc = white[0]
    on_pos = white[1]
    to_pos = white[2:]
    which_one = {'K':king , 'Q':queen , 'N':knight , 'B':bishop , 'R' : rook}
    for piece, from_ in white_pieces.items():
      if piece.startswith(pc) and from_.startswith(on_pos):
        if which_one[pc](from_ , to_pos):
          white_pieces[piece] = to_pos
          break
    return(white_pieces)

WHITE AND BLACK CAPTURE

In [0]:
def captures_white(move , white_pawns , white_pieces , black_pawns , black_pieces):
  import re
  pat = re.compile(r'^[a-h]x')
  pat1 =re.compile(r'^[BQRNKQ]x')
  if (pat.match(move)):
    for pawn, from_ in white_pawns.items():
      if from_.startswith(move[0]):
        if wPawncap(from_ , move[2:]):
          white_pawns[pawn] = move[2:]
    try:
      temp = list(black_pawns.keys())[list(black_pawns.values()).index(move[2:])]
      del black_pawns[temp]
    except:
      temp = list(black_pieces.keys())[list(black_pieces.values()).index(move[2:])]
      del black_pieces[temp]
  elif pat1.match(move):
     which_one = {'K':king , 'Q':queen , 'N':knight , 'B':bishop , 'R' : rook}
    #3print(pc, white)
     for piece , from_ in white_pieces.items():
       if piece.startswith(move[0]):
         if which_one[move[0]](from_ , move[2:]):
           white_pieces[piece] = move[2:]
           break
     try:
      temp = list(black_pawns.keys())[list(black_pawns.values()).index(move[2:])]
      del black_pawns[temp]
     except:
       temp = list(black_pieces.keys())[list(black_pieces.values()).index(move[2:])]
       del black_pieces[temp]

  return(white_pawns , white_pieces , black_pawns , black_pieces)



def captures_black(move , white_pawns , white_pieces , black_pawns , black_pieces):
  import re
  pat = re.compile(r'^[a-h]x')
  pat1 =re.compile(r'^[BQRNKQ]x')
  if (pat.match(move)):
    for pawn, from_ in black_pawns.items():
      if from_.startswith(move[0]):
        if bPawncap(from_ , move[2:]):
          black_pawns[pawn] = move[2:]
    try:
      temp = list(white_pawns.keys())[list(white_pawns.values()).index(move[2:])]
      del white_pawns[temp]
    except:
      temp = list(white_pieces.keys())[list(white_pieces.values()).index(move[2:])]
      del white_pieces[temp]
  elif pat1.match(move):
     which_one = {'K':king , 'Q':queen , 'N':knight , 'B':bishop , 'R' : rook}
    #3print(pc, white)
     for piece , from_ in black_pieces.items():
       if piece.startswith(move[0]):
         if which_one[move[0]](from_ , move[2:]):
           black_pieces[piece] = move[2:]
           break
     try:
      temp = list(white_pawns.keys())[list(white_pawns.values()).index(move[2:])]
      del white_pawns[temp]
     except:
       temp = list(white_pieces.keys())[list(white_pieces.values()).index(move[2:])]
       del white_pieces[temp]

  return(white_pawns , white_pieces , black_pawns , black_pieces)


AMBIGUOUS CAPTURE

In [0]:
def ambiguous_white_capture(white , white_pieces , black_pieces, black_pawns):
     which_one = {'K':king , 'Q':queen , 'N':knight , 'B':bishop , 'R' : rook}
     for piece , from_ in white_pieces.items():
       if piece.startswith(white[0]) and from_[0] == white[1]:
         if which_one[white[0]](from_ , white[3:]):
           white_pieces[piece] = white[3:]
           break
     try:
      temp = list(black_pawns.keys())[list(black_pawns.values()).index(white[3:])]
      del black_pawns[temp]
     except:
       temp = list(black_pieces.keys())[list(black_pieces.values()).index(white[3:])]
       del black_pieces[temp]

     return( white_pieces , black_pawns, black_pieces)

def ambiguous_black_capture(black , black_pieces , white_pieces, white_pawns):
     which_one = {'K':king , 'Q':queen , 'N':knight , 'B':bishop , 'R' : rook}
     for piece , from_ in black_pieces.items():
       if piece.startswith(black[0]) and from_[0] == black[1]:
         if which_one[black[0]](from_ , black[3:]):
           black_pieces[piece] = black[3:]
           break
     try:
      temp = list(white_pawns.keys())[list(white_pawns.values()).index(black[3:])]
      del white_pawns[temp]
     except:
       temp = list(white_pieces.keys())[list(white_pieces.values()).index(black[3:])]
       del white_pieces[temp]

     return( white_pawns , white_pieces , black_pieces)

PAWN PROMOTION

In [0]:
import re
def pawn_promotion(move , white_pawns , white_pieces, black_pawns, black_pieces):
  temp = ''
  if '(' and ')' in move:
    move = move.replace(')' , '').replace('(' , '')
  elif '=' in move:
    move = move.replace('=' , '')
  if int(move[1]) < 4:
    for pawn , from_ in black_pawns.items():
       if bPawn(from_ , move[:2]) or bPawncap(from_ , move[:2]):
         temp = pawn
         break
    del black_pawns[temp]
    to_add = ''
    if move[2] in 'NBRQ':
      to_add = move[2] + str(list(black_pieces.keys()).count(move[2]))
    white_pieces[to_add] = move[:2]
    
  elif int(move[1]) >= 4:
     for pawn , from_ in white_pawns.items():
       if wPawn(from_ , move[:2]) or wPawncap(from_ , move[:2]):
         temp = pawn
         break
     del white_pawns[temp]
     to_add = ''
     if move[2] in 'NBRQ':
       to_add = move[2] + str(list(white_pieces.keys()).count(move[2]))
     white_pieces[to_add] = move[:2]
    
  return(white_pawns , white_pieces , black_pawns , black_pieces)
  


In [0]:
import re
ambiguous_move = re.compile('^[NBR]..[0-8]$')
ambiguous_capture = re.compile('^[NBR][a-h]x.[0-8]$')

def update(white ,black, white_pawns ,white_pieces , black_pawns, black_pieces):
 
    if '+' in white:
      white = white[:-1]

    if '+' in black:
     black = black[:-1]
    
    if '=' in white or '(' and ')' in white:
      white_pawns , white_pieces , black_pawns , black_pieces = pawn_promotion(white ,  white_pawns , white_pieces , black_pawns , black_pieces) 
    
    if 'x' in white:
      if ambiguous_capture.match(white):
        white_pieces, black_pawns , black_pieces = ambiguous_white_capture(white , white_pieces , black_pieces, black_pawns)
      else:
        white_pawns ,white_pieces , black_pawns , black_pieces = captures_white(white , white_pawns , white_pieces , black_pawns , black_pieces)

    elif white == 'O-O':
      white_pieces = white_kings_castle(white_pieces)

    elif white == 'O-O-O':
      white_pieces = white_queens_castle(white_pieces)
      
    elif re.match(ambiguous_move , white):
      white_pieces = white_ambiguous(white , white_pieces)
    
    elif white[0].islower():
      white_pawns = white_pawn_moves(white , white_pawns)

    elif white[0].isupper():
      white_pieces = white_piece_moves(white , white_pieces)


    if black == '':
      pass


    elif '=' in black or '(' and ')' in black:
      white_pawns , white_pieces , black_pawns , black_pieces = pawn_promotion(black ,  white_pawns , white_pieces , black_pawns , black_pieces) 


    elif 'x' in black:
      if ambiguous_capture.match(black):
        white_pawns , white_pieces , black_pieces = ambiguous_black_capture(black , black_pieces , white_pieces, white_pawns)
      else:
        white_pawns ,white_pieces , black_pawns , black_pieces = captures_black(black, white_pawns , white_pieces , black_pawns , black_pieces)

    elif black == 'O-O':
      black_pieces = black_kings_castle(black_pieces)

    elif black == 'O-O-O':
      black_pieces = black_queens_castle(black_pieces)
      
    elif re.match(ambiguous_move , black):
      black_pieces = black_ambiguous(black , black_pieces)
    
    elif black[0].islower():
      black_pawns = black_pawn_moves(black , black_pawns)

    elif black[0].isupper():
      black_pieces = black_piece_moves(black , black_pieces)
    
    return (white_pawns, white_pieces, black_pawns, black_pieces)


In [0]:
def final_positions():  
  
  white_pieces = (dict(zip(['R1' ,'N1' ,'B1' ,'Q' ,'K','B2','N2','R2'], [x+y for x in 'abcdefgh' for y in '1'])))
  white_pawns = (dict(zip(['P1' ,'P2' ,'P3' ,'P4' ,'P5','P6','P7','P8'], [x+y for x in 'abcdefgh' for y in '2'])))
  black_pieces = (dict(zip(['R1' ,'N1' ,'B1' ,'Q' ,'K','B2','N2','R2'], [x+y for x in 'abcdefgh' for y in '8'])))
  black_pawns = (dict(zip(['P1' ,'P2' ,'P3' ,'P4' ,'P5','P6','P7','P8'], [x+y for x in 'abcdefgh' for y in '7'])))

  for moves in game[:]:
    white , black = moves[0] , moves[1]
    white_pawns ,white_pieces , black_pawns, black_pieces = update(white, black, white_pawns ,white_pieces , black_pawns, black_pieces)
  return("White_pawns" ,white_pawns , "White_pieces" ,white_pieces , "Black_pawns", black_pawns, "Black_pieces" , black_pieces)


WHITE_PAWNS = (final_positions()[0] , final_positions()[1])
WHITE_PIECES = (final_positions()[2] , final_positions()[3])
BLACK_PAWNS = (final_positions()[4] , final_positions()[5])
BLACK_PIECES = (final_positions()[6] , final_positions()[7])
print(WHITE_PAWNS)
print(WHITE_PIECES)
print(BLACK_PAWNS) 
print(BLACK_PIECES)


('White_pawns', {'P2': 'c5', 'P6': 'f2', 'P7': 'g2', 'P8': 'h5'})
('White_pieces', {'R1': 'd6', 'B1': 'g5', 'Q': 'f5', 'K': 'g1'})
('Black_pawns', {'P1': 'a7', 'P2': 'b6', 'P5': 'e5', 'P6': 'f7', 'P8': 'h6'})
('Black_pieces', {'Q': 'c7', 'K': 'g7', 'N2': 'd7', 'R2': 'd8'})
